In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
#import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from efficientnet_pytorch import EfficientNet

from pytorchcv.model_provider import get_model
import gc
import timm


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
class MetaModel(nn.Module):
    def __init__(self, models, device='cuda:0', extended=False):
        super(MetaModel, self).__init__()
        
        self.extended = extended
        self.device = device
        self.models = models
        self.len = len(models)
        
        if self.extended:
            self.bn = nn.BatchNorm1d(self.len)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.2)

        self.fc = nn.Linear(self.len, 1)
        
    def forward(self, x):
        x = torch.cat(tuple([model(x) for model in self.models]), dim=1)
        
        if self.extended:
            x = self.bn(x)
            x = self.relu(x)
            #x = self.dropout(x)
            
        x = self.fc(x)
        
        return x

In [3]:
def disable_grad(model):
    for parameter in model.parameters():
        parameter.requires_grad = False
        
    return model

def validate(model, x_val, batch_size, checkpoint=0.31):
    val_loss = []
    dataloader_iterator = iter(x_val)
    model.eval()
    with torch.no_grad():
        for batch_idx in tqdm.tqdm_notebook(range(len(x_val))):
            try:
                X_batch, y_batch, _ = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_val)
                X_batch, y_batch, _ = next(dataloader_iterator)

            y_batch = torch.FloatTensor(y_batch).to(device)

            test_preds = model(X_batch.to(device))
            test_loss_value = F.binary_cross_entropy_with_logits(test_preds, y_batch).item() * batch_size
            #test_loss_value /= len(models)

            val_loss.append(test_loss_value)

        mean_loss = sum(val_loss) / (len(x_val) * batch_size)
        
        if mean_loss <= checkpoint:
            torch.save(model.state_dict(), model.__class__.__name__  + ' ' + str(mean_loss) + '.pth')

        print('Validation: ', mean_loss)
        
        return mean_loss
        
def train(model, x_train, x_val, optimizer, scheduler, batch_size, epochs=10):
    test_loss_history = []

    for epoch in tqdm.tqdm_notebook(range(epochs)):
        dataloader_iterator = iter(x_train)
        model.train()
        
        train_loss = []

        for batch_idx in tqdm.tqdm_notebook(range(len(x_train))):   
            try:
                X_batch, y_batch, _ = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_train)
                X_batch, y_batch, _ = next(dataloader_iterator)

            y_batch = torch.FloatTensor(y_batch).to(device)
            
            if len(y_batch) > 0 and len(X_batch) == batch_size and len(X_batch) == len(y_batch):
                optimizer.zero_grad()

                preds = model(X_batch.to(device))

                loss_value = F.binary_cross_entropy_with_logits(preds, y_batch)
                loss_value.backward()

                optimizer.step()
                #scheduler.step()

                train_loss.append(loss_value.item() * batch_size)
            else:
                print("Unable to make an epoch")

        scheduler.step()
        test_loss_value = validate(model, x_val, batch_size)

        mean_loss = sum(train_loss) / (len(x_train) * batch_size)
        
        print('Train: ', mean_loss)
        print('Expected LB value:', test_loss_value + 0.08228, test_loss_value * 100 / 90)
        print('Epoch:', epoch+1)

        test_loss_history.append(test_loss_value)

    gc.collect()

    return test_loss_history

def extract_features_to_csv(model, x_val, batch_size, out_file):
    dataloader_iterator = iter(x_val)
    model.eval()
    with torch.no_grad():
        for batch_idx in tqdm.tqdm_notebook(range(len(x_val))):
            try:
                X_batch, y_batch = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_val)
                X_batch, y_batch = next(dataloader_iterator)

            features = model(X_batch.to(device))
            #features = features.view(features.size(0), features.size(1) * features.size(2) * features.size(3))
            
            with open(out_file, 'a') as w:
                for i in range(batch_size):
                    w.write('\n' + ','.join([str(i) for i in list(features[i].cpu().numpy())]) + ',' + str(y_batch[i].item()))
                    
                    
def remove_noisy_images(model, x_val, batch_size):
    dataloader_iterator = iter(x_val)
    
    model.eval()
    to_move = []
    with torch.no_grad():
        for batch_idx in tqdm.tqdm_notebook(range(len(x_val))):
            try:
                X_batch, y_batch, paths = next(dataloader_iterator)
            except:
                dataloader_iterator = iter(x_val)
                X_batch, y_batch, paths = next(dataloader_iterator)
                
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            preds = torch.sigmoid(model(X_batch))
            try:
                for i in range(len(preds)):
                    pred = preds[i].item()
                    
                    if 0.6 >= pred >= 0.4:
                        path = paths[i]
                        label = 'fake' if y_batch[i].item() else 'real'
                        to_move.append([path, 'data\\' + label + '\\' + path.split('\\')[-1]])
                        #os.rename(path, 'data\\' + label + '\\' + path.split('\\')[-1])
                        
            except Exception as e:
                print(str(e))
                
    return to_move

In [4]:
y_train = pd.read_csv(r'data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [5]:
batch_size = 40
x_val = load_img_val_dataset(r'data\img_val', batch_size)
x_train = load_img_dataset(r'data\least', batch_size, resize=256, crop=None, num_samples=30000)

In [6]:
#%%script false

models = []
weights = []
raw_models = \
[
    ['pretrained/metav2/efficientnetb2 0.8548137313946486 0.3376769562025044.pth', 'efficientnet-b2'],
    ['pretrained/metav2/EfficientNetb3 0.8573518024606384 0.34558522378585194.pth', 'efficientnet-b3'],
    ['pretrained/metav2/EfficientNetb4 0.8579110384582294 0.3383911053075265.pth', 'efficientnet-b4'],
    ['pretrained/metav2/EfficientNet6 0.8602770369095758 0.33193617861157143.pth', 'efficientnet-b6'],
    
    ['pretrained/metav1/EfficientNetb0 t2 0.8616966359803837 0.3698434531609828.pth', 'efficientnet-b0'],
    ['pretrained/metav1/EfficientNetb1 t2 0.8410909403768391 0.36058002083572327.pth', 'efficientnet-b1'],
    ['pretrained/metav1/EfficientNetb2 t2 0.8659554331928073 0.35598630783834084.pth', 'efficientnet-b2'],
    ['pretrained/metav1/EfficientNetb3 t2 0.8486191172674868 0.3611779548592305.pth', 'efficientnet-b3']
]
meta_models = \
[
    ['pretrained/metav1/MetaModel 0.30638167556896007.pth', raw_models[4:], False],
    ['pretrained/metav2/MetaModel 0.2919331893755284.pth', raw_models[:4], False],
    ['pretrained/metav3 (1+2)/MetaModel 0.30281482560578044.pth', raw_models, True]
]


for meta_data in meta_models:
    models_for_meta = []
    
    for raw_data in meta_data[1]:
        model = EfficientNet.from_pretrained(raw_data[1], num_classes=1)
        model.load_state_dict(torch.load(raw_data[0], map_location=device))
        _ = model.eval()
        _ = disable_grad(model)
        model = model.to(device)
        models_for_meta.append(model)

        del model
        
    model = MetaModel(models_for_meta, extended=meta_data[2]).to(device)
    model.load_state_dict(torch.load(meta_data[0], map_location=device))
    _ = model.eval()
    _ = disable_grad(model)
    model = model.to(device)
    models.append(model)
    
    del model

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b6
Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3


In [7]:
epochs = 10
model = MetaModel(models, extended=False).to(device)
#model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
#model.load_state_dict(torch.load('EfficientNetb0 0.7783704723393272 0.8247214939030197.pth'))
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(x_train), epochs=epochs)

In [8]:
train(model, x_train, x_val, optimizer, scheduler, batch_size, epochs=epochs)
#validate(model, x_val, batch_size, checkpoint=0.0)

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



Validation:  0.27042650912737926
Train:  0.4189150613149007
Expected LB value: 0.3527065091273793 0.3004738990304214
Epoch: 1


KeyboardInterrupt: 

In [ ]:
%%script false

files = remove_noisy_images(model, x_train, batch_size)

for file in files:
    try:
        os.rename(file[0], file[1])
    except Exception as e:
        print(str(e))

#torch.save(model.state_dict(), model.__class__.__name__  + '.pth')